# Script Teste

_Script_ que define a classe `teste_cepespdata`

In [1]:
import pandas as pd
import numpy as np
import requests
import os

## Função de Testes Específicos

In [78]:
## Votos Brancos
def teste_votos_brancos_nulos(banco):
    qtde_votos_brancos = banco[banco.numero_candidato == 95].qtde_votos.sum()
    qtde_votos_nulos = banco[banco.numero_candidato == 96].qtde_votos.sum()
    qtde_votos_total = banco.qtde_votos.sum()
    brancos = qtde_votos_brancos / qtde_votos_total
    nulos = qtde_votos_nulos / qtde_votos_total
    return {'brancos':brancos, 'nulos':nulos}

## Votos Legenda
# Proporção de votos legenda
def teste_voto_legenda(banco):
    numero_legendas = range(10,91) # partidos entre 10 e 90 inclusivo
    
    qtde_votos_legenda = banco[np.in1d(banco.numero_candidato, numero_legendas)].qtde_votos.sum()
    qtde_votos_total = banco.qtde_votos.sum()

    prop_votos_legenda = qtde_votos_legenda / qtde_votos_total

    return {'votos_legenda': prop_votos_legenda}

## N de Cidades
def teste_cidades(banco):
    n_unique_ibge = len(set(banco.cod_mun_ibge))
    n_unique_tse = len(set(banco.cod_mun_tse))

    return {'ibge': n_unique_ibge, 'tse': n_unique_tse}

## CPF e Títulos
# Proporção de CPF e Títulos com número correto
def teste_cpf_titulos(banco):
    len_titulo = [(lambda x: len(str(x)))(x) for x in banco.num_titulo_eleitoral_candidato]
    len_cpf = [(lambda x: len(str(x)))(x) for x in banco.cpf_candidato]
    prop_titulo = sum(np.array(len_titulo) == 12) / len(len_titulo)
    prop_cpf    = sum(np.array(len_cpf) == 11) / len(len_cpf)
    return {'prop_titulo': prop_titulo, 'prop_cpf': prop_cpf}

def teste_votos(banco, agregacao_regional):
    molde = pd.read_csv("molde_secao.csv")
    molde[(molde.ANO_ELEICAO == self.ano) & (molde.CODIGO_CARGO == self.cargo)][["COMPARECIMENTO"]]
    banco[['ano_eleicao', 'uf', 'codigo_cargo','qtde_votos']]

In [83]:
class teste_cepespdata:
    
    ## Construtor
    def __init__(self, ano, cargo, agregacao_politica = 2, agregacao_regional = 0, test = False):
        
        if test:
            self.u0 = "http://test.cepesp.io/api/consulta/"
        else:
            self.u0 = 'http://cepesp.io/api/consulta/'
            
        self.ano = str(ano)
        self.cargo = str(cargo)
        self.agregacao_politica = str(agregacao_politica)
        self.agregacao_regional = str(agregacao_regional)
        self.log = {'ano':self.ano,
                    'cargo':self.cargo,
                    'agregacao_politica': self.agregacao_politica,
                    'agregacao_regional': self.agregacao_regional,
                    'banco_candidatos':{},
                    'banco_eleicoes':{}}
        self.download_candidatos()
        self.download_eleicoes()
    
    ## Download dos Bancos
    def download_eleicoes(self):
        colunas = 'c[]=ANO_ELEICAO&c[]=NUM_TURNO&c[]=COD_MUN_TSE&c[]=COD_MUN_IBGE&c[]=NOME_MUNICIPIO&c[]=UF&c[]=CODIGO_CARGO&c[]=DESCRICAO_CARGO&c[]=NOME_CANDIDATO&c[]=NUMERO_CANDIDATO&c[]=CPF_CANDIDATO&c[]=DES_SITUACAO_CANDIDATURA&c[]=NUMERO_PARTIDO&c[]=SIGLA_PARTIDO&c[]=NUM_TITULO_ELEITORAL_CANDIDATO&c[]=QTDE_VOTOS'
        u0 = self.u0 + "/tse?format=csv&cargo=" + \
                self.cargo + "&anos[]=" + \
                self.ano+"&agregacao_regional=" + \
                self.agregacao_regional+"&agregacao_politica=" + \
                self.agregacao_politica+"&brancos=1&nulos=1&" +\
                colunas
        
        eleicoes_df = pd.read_csv(u0, sep = ",", dtype = {'num_titulo_eleitoral_candidato': str,
                                                          'cpf_candidato': str})
        self.banco_eleicoes = eleicoes_df
    
    ## Definição de Testes Específicos
    
    def teste_eleitos(self):
        banco = self.banco_candidatos
        valores_eleitos = "ELEITO", 'ELEITO POR QP', 'ELEITO POR MÉDIA'
        qtde_eleitos = sum(np.in1d(x.banco_candidatos.DESC_SIT_TOT_TURNO, valores_eleitos))
        molde = pd.read_csv("data/moldes/molde_vagas.csv")
        vagas = molde[(molde.ANO_ELEICAO == int(self.ano)) & (molde.CODIGO_CARGO == int(self.cargo))][['QTDE_VAGAS']].sum()[0]
        
        return {'vagas_esperadas': vagas, 'qtde_eleitos': qtde_eleitos}
    
    
    ## Chamadas de Testes Gerais
    def download_candidatos(self):
        u0 = self.u0 + "candidatos?format=csv&cargo="+self.cargo+"&anos[]="+self.ano
        candidatos_df = pd.read_csv(u0, sep = ',')
        self.banco_candidatos = candidatos_df
        
    def teste_eleicoes(self):    
        b1 = 'banco_eleicoes'
        self.log[b1] = {}
        self.log[b1]['brancos_nulos'] = teste_votos_brancos_nulos(self.banco_eleicoes)
        self.log[b1]['cpf_titulo'] = teste_cpf_titulos(self.banco_eleicoes)
        self.log[b1]['votos_legenda'] = teste_voto_legenda(self.banco_eleicoes)
        if self.agregacao_regional == 6: # Teste n de cidades apenas se agreg. reg. for municipal
            self.log[b1]['n_cidades'] = teste_cidades(self.banco_eleicoes)
            
    def teste_candidatos(self):
        b1 = 'banco_candidatos'
        self.log[b1]['qtde_eleitos'] = self.teste_eleitos()
                
    def teste_geral(self):
        print("Starting test...\nyear: " + self.ano + "\nposition: " + self.cargo)
        
        self.teste_eleicoes()
        self.teste_candidatos()
        
    def __str__(self):
        texto = '''
ANO: {}, CARGO: {}

** BANCO ELEICOES - BETA

Proporção Branco: {:.2f}
Proporção Nulos: {:.2f}

Proporção de CPF com 11 dígitos: {:.2f}
Proporção de títulos com 12 dígitos: {:.2f}

Proporção de votos em legenda: {:.2f}

** BANCO CANDIDATOS

Vagas: {}
Eleitos: {}
'''
        banco_eleicoes = self.log['banco_eleicoes']
        banco_candidatos = self.log['banco_candidatos']
        print_out = texto.format(self.log['ano'], 
                                 self.log['cargo'],
                                 banco_eleicoes['brancos_nulos']['brancos'],
                                 banco_eleicoes['brancos_nulos']['nulos'],
                                 banco_eleicoes['cpf_titulo']['prop_cpf'],
                                 banco_eleicoes['cpf_titulo']['prop_titulo'],
                                 banco_eleicoes['votos_legenda']['votos_legenda'],
                                 banco_candidatos['qtde_eleitos']['vagas_esperadas'],
                                 banco_candidatos['qtde_eleitos']['qtde_eleitos'])
        return print_out

In [84]:
x = teste_cepespdata(2014,1, agregacao_regional=6)

In [85]:
x.teste_geral()

Starting test...
year: 2014
position: 1


In [86]:
print(x)


ANO: 2014, CARGO: 1

** BANCO ELEICOES - BETA

Proporção Branco: 0.03
Proporção Nulos: 0.05

Proporção de CPF com 11 dígitos: 0.74
Proporção de títulos com 12 dígitos: 0.74

Proporção de votos em legenda: 0.92

** BANCO CANDIDATOS

Vagas: 1
Eleitos: 1

